# ANATEL TF-IDF Recomendação Baseada em Conteúdo

Este código utiliza o TF-IDF (Term Frequency Inverse Document Frequency) para realizar a recomendação de resoluções da Anatel a partir de uma determinada resolução.

O algoritmo retorna as "top k" resoluções similares as demais resoluções do arquivo CSV que contém as resoluções.

In [ ]:
# Set o path do ambiente para encontrar o Recommenders
import sys
sys.path.append("../../")

# Import functions
from reco_utils.recommender.tfidf.tfidf_utils import TfidfRecommender

# Import Pandas
import pandas as pd

# Print version
print("System version: {}".format(sys.version))

### 1. Carrega o dataset no dataframe

In [ ]:
# Declara a váriavel filename com o nome do arquivo que contém as resoluções
filename = 'resolucoes_text_2.csv'

# Lê o arquivo, o qual está separado por '\\'
df = pd.read_csv(filename, sep='\\')

Verifica o tamanho do arquivo e excluí as colunas nulas 'nan'

In [ ]:
# Preview metadata for public domain articles
print('tamanho: ' + str(len(df)))
df.drop('Unnamed: 1', inplace=True, axis=1)
df.drop('Unnamed: 3', inplace=True, axis=1)
df.drop('Unnamed: 4', inplace=True, axis=1)
df

### 4. Instancia o recommender
Todas as funcções para a preparação dos dados e das recomendações estão na classe **TfidfRecommender** a qual foi importada. Antes de executar estas funções, é necessário criar um objeto desta classe. 

Selecione um dos seguintes métodos de tokenização (tokenization) para utilizar no modelo:

| tokenization_method | Descrição                                                                                                                      |
|:--------------------|:---------------------------------------------------------------------------------------------------------------------------------|
| 'none'              | Nenhuma tokenização é aplicada. Cada palavra é considerada um token.                                                                     |
| 'nltk'              | Simple stemming é aplicado usando NLTK.                                                                                           |
| 'bert'              | HuggingFace BERT word tokenization ('bert-base-cased') é aplicado.                                                               |
| 'scibert'           | SciBERT word tokenization ('allenai/scibert_scivocab_cased') é aplicado.<br> Este método é indicado para artigos científicos. |

In [ ]:
# Cria o objeto recommender
recommender = TfidfRecommender(id_col='id_resolucao', tokenization_method='scibert')

### 5. Prepara o texto para uso no TF-IDF recommender

Verificando o texto de uma resolução como exemplo.

In [ ]:
# Preview the first 1000 characters of the full scientific text from one example
print(df['Texto'][0][:1000])

Nos textos das resoluções há alguns caracteres especiais (tais como • ▲ ■ ≥ °) e pontuação que dever ser removidas antes de incluir o text no dataframe. Letras maiúsculas são mantidas para o [BERT-based tokenization methods](https://huggingface.co/transformers/model_doc/bert.html), mas é removida para simple ou no tokenization.


In [ ]:
# Designa a coluna na qual será realizada a limpeza
cols_to_clean = ['Texto']
clean_col = 'cleaned_text'
df_clean = recommender.clean_dataframe(df, cols_to_clean, clean_col)

In [ ]:
# Faz um preview do dataframe com o texto limpo
df_clean.head()

In [ ]:
# Faz um preview dos primeiros 1000 caracteres do campo cleaned_text
print(df_clean[clean_col][0][:1000])

Realiza a tokenização do texto limpo (cleaned_text)para utilizar no modelo TF-IDF. Os tokens são armazenados no objeto TfidfRecommender.

In [ ]:
# Realiza a tokenização do texto com o método definido na classe instanciada
tf, vectors_tokenized = recommender.tokenize_text(df_clean, text_col=clean_col)

### 6. Recomenda as resoluções utilizando o TF-IDF


In [ ]:
# Ajusta a vetorização do TF-IDF
recommender.fit(tf, vectors_tokenized)

# Recebe as recomendações
top_k_recommendations = recommender.recommend_top_k_items(df_clean, k=2)

In our recommendation table, each row represents a single recommendation.

- **cord_uid** corresponds to the article that is being used to make recommendations from.
- **rec_rank** contains the recommdation's rank (e.g., rank of 1 means top recommendation).
- **rec_score** is the cosine similarity score between the query article and the recommended article.
- **rec_cord_uid** corresponds to the recommended article.

In [ ]:
# Visualiza as recomendações
top_k_recommendations